In [1]:
%load_ext rpy2.ipython

/home/fpavlov/miniconda3/envs/mirna/lib/python3.11/site-packages/rpy2/ipython/rmagic.py:83: UserWarning: The Python package `pandas` is strongly recommended when using `rpy2.ipython`. Unfortunately it could not be loaded, as we did not manage to load `numpy` in the first place (error: No module named 'numpy').
  warnings.warn('The Python package `pandas` is strongly '


In [2]:
%%R

library(GenomicFeatures)
library(ChIPpeakAnno)
library(ChIPseeker)
library(clusterProfiler)
library(dplyr)
library(stringr)

R[write to console]: Error in library(GenomicFeatures) : 
  there is no package called ‘GenomicFeatures’




Error in library(GenomicFeatures) : 
  there is no package called ‘GenomicFeatures’


RInterpreterError: Failed to parse and evaluate line '\nlibrary(GenomicFeatures)\nlibrary(ChIPpeakAnno)\nlibrary(ChIPseeker)\nlibrary(clusterProfiler)\nlibrary(dplyr)\nlibrary(stringr)\n'.
R error message: 'Error in library(GenomicFeatures) : \n  there is no package called ‘GenomicFeatures’'

In [7]:
import pandas as pd
from pathlib import Path

d_root = Path("/home/fpavlov/projects/article_conserved_miRNA")
d_data = d_root / "data"

# output
f_mm_v25_annotation = d_data / "gencode.vM25.annotation.gtf.gz"
f_mm_v25_annotation_str = str(d_data / "gencode.vM25.annotation.gtf.gz")
f_flipon_to_gene_anno = d_data / "flipon_to_gene_anno.tsv"


In [ ]:
!wget -c -O {f_mm_v25_annotation} -q --show-progress http://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_mouse/release_M25/gencode.vM25.annotation.gtf.gz

/home/fpavlov/proje 100%[===================>]  27,22M   997KB/s    in 37s     


In [ ]:
%R -i f_mm_v25_annotation_str

UsageError: Line magic function `%R` not found.


In [ ]:
# %%R

# txdb <- makeTxDbFromGFF(f_mm_v25_annotation_str)

# annoData <- toGRanges(txdb, format='gene')
# annoData[1:2]

In [ ]:
%%R
txdb_all_annotation <- makeTxDbFromGFF(f_mm_v25_annotation_str)

UsageError: Cell magic `%%R` not found.


In [ ]:
%%R

# https://www.biostars.org/p/140471/
gtf.file = f_mm_v25_annotation_str
gtf.gr = rtracklayer::import(gtf.file) # creates a GRanges object
gtf.df = as.data.frame(gtf.gr)

genes = unique(gtf.df[ ,c("gene_id","gene_name")])
# genes

UsageError: Cell magic `%%R` not found.


In [ ]:
ssdna_list = [
    str(d_data / "mm10_kouzine_ssDNA_bed" / "mm10_kouzine_actb_ssdna_enriched_z-dna.bed"),
    str(d_data / "mm10_kouzine_ssDNA_bed" / "mm10_kouzine_actb_ssdna_enriched_quadruplex.bed"),
    str(d_data / "mm10_kouzine_ssDNA_bed" / "mm10_kouzine_actb_ssdna_enriched_sidd.bed"),
    str(d_data / "mm10_kouzine_ssDNA_bed" / "mm10_kouzine_actb_ssdna_enriched_h-dna.bed"),
]

%R -i ssdna_list

UsageError: Line magic function `%R` not found.


In [ ]:
%%R

df_final <- data.frame()

for (ssdna in ssdna_list) {
  # print('--------------------------------------------')
  # print(ssdna)
  # print('--------------------------------------------')
  peakAnno <- annotatePeak(ssdna, tssRegion=c(-3000, 3000), TxDb=txdb_all_annotation)
  # print(peakAnno)

  anno_df <- data.frame(peakAnno@anno)
  # res_df <- anno_df[grepl("Promoter", anno_df$annotation),]
  res_df <- anno_df[c('V4', 'seqnames','start','end','geneStrand','annotation','geneId','transcriptId')]
  # res_df <- anno_df[grepl("Promoter", anno_df$annotation),][c('annotation', 'geneId')]
  # res_df <- distinct(res_df, geneId, .keep_all=TRUE)
  res_df$group <- str_replace(str_split(ssdna, 'enriched_')[[1]][2], '.bed', '')
  res_df <- merge(res_df, genes, by.x='geneId', by.y='gene_id')

  df_final <- bind_rows(df_final, res_df)
  
  # print(dim(res_df))
}

# write.table(df_final, 'flipon_to_gene_anno.tsv', append=FALSE, sep="\t", dec=".", row.names=FALSE, col.names = TRUE)
# df_final


UsageError: Cell magic `%%R` not found.


In [ ]:
df_final = %R df_final

df_final = df_final[['V4', 'seqnames', 'start', 'end', 'geneId', 'gene_name', 'transcriptId', 'geneStrand', 'annotation', 'group']]

df_final.to_csv(f_flipon_to_gene_anno, sep='\t', index=False)
df_final

UsageError: Line magic function `%R` not found.


In [ ]:
# chr13 78,198,426 - 78,198,445


In [9]:
df_final = pd.read_table(f_flipon_to_gene_anno)

df_final = df_final[df_final['V4']=="chr13:78198426-78198445"]

df_final

,V4,seqnames,start,end,geneId,gene_name,transcriptId,geneStrand,annotation,group
17461,chr13:78198426-78198445,13,78198427,78198445,ENSMUSG00000087143.9,A830082K12Rik,ENSMUST00000145613.7,1,Promoter (<=1kb),z-dna
